In [2]:
import pandas as pd
import logging

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO, filename="Revenue_by_city_region_logging.log",filemode="w")



payments= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\payment.csv')
orders= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\orders.csv')

cafes=pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\cafe.csv')
address=pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\address.csv')

merged_df=payments.merge(orders,on='payment_id',how='left',indicator=True)

payment_date_converted=pd.DatetimeIndex(merged_df['payment_date']) #convert from object to datetime payment_date_converte

merged_df['Year']=payment_date_converted.year
merged_df['Month']=payment_date_converted.month  

merged_df=merged_df.sort_values(by='payment_date',ascending=True)

merged_df=merged_df.drop_duplicates(subset='payment_id',keep='first')#removing duplicates with same payment_id
columns_todrop=['customer_id_x','emp_id','created_at','payment_type', #removing columns that are not needed
                'order_id','item_id','customer_id_y','delivery',
                'delivery_address_id','item_quantity',
                'customer_modifications','delivery_notes','_merge']
merged_df=merged_df.drop(columns=columns_todrop,axis=1)

columns=['cafe_id','payment_date','Year','Month','payment_id','amount']
merged_df=merged_df[columns]
merged_df['payment_date']=pd.to_datetime(merged_df['payment_date'])
merged_df['Year']=merged_df['Year'].astype('str')
merged_df['Month']=merged_df['Month'].astype('str')
#using .loc to change all values in column 'Month' to 2 digits
merged_df['Month'] = merged_df['Month'].str.zfill(2)
merged_df['Year_Month']=merged_df['Year']+'-'+merged_df['Month']

#rearranging columns and ordering by cafe_id and Year_Month
first_column = merged_df.pop('Year_Month') 
merged_df.insert(0, 'Year_Month', first_column)
first_column = merged_df.pop('cafe_id') 
merged_df.insert(0, 'cafe_id', first_column)

#calculating monthly revenue by location
merged_df=merged_df.sort_values(by=['cafe_id','Year_Month'],ascending=True)

cafe_locations_monthly_df=merged_df.groupby(['cafe_id','Year_Month','Year','Month']).agg(
    monthly_revenue_total=('amount','sum')
).reset_index()

cafe_locations_monthly_df=cafe_locations_monthly_df.reset_index(drop=True)

cafe_address_df=pd.merge(cafes,address,on='address_id',how='left')

revenue_df=pd.merge(cafe_locations_monthly_df,cafe_address_df,on='cafe_id',how='left')
revenue_df=revenue_df.reset_index(drop=True)

revenue_df=revenue_df.drop(columns=['manager_emp_id','phone_number','address_id','last_update_x','address_first_line',
                         'address_id','address_second_line','postal_code','last_update_y','city'],axis=1)


revenue_df=revenue_df.loc[:,['country','Year_Month','Year','monthly_revenue_total']]

#aggregating monthly revenue by country
monthly_revenue_by_country_df=revenue_df.groupby(['country','Year_Month','Year']).agg(
    total_monthly_revenue=('monthly_revenue_total','sum'),
    avg_revenue_for_this_month_of_the_year=('monthly_revenue_total','mean')
)

monthly_revenue_by_country_df['running_total_by_month']=monthly_revenue_by_country_df.groupby(['country','Year'])['total_monthly_revenue'].cumsum()
monthly_revenue_by_country_df=monthly_revenue_by_country_df.reset_index()

region_dictionary={"USA":"North America","Peru":"LATAM","Argentina":"LATAM"}
monthly_revenue_by_country_df['region']=monthly_revenue_by_country_df['country'].map(region_dictionary)

monthly_revenue_by_country_df=monthly_revenue_by_country_df.loc[:,['region','country','Year','Year_Month','total_monthly_revenue','avg_revenue_for_this_month_of_the_year' ,'running_total_by_month']]

monthly_revenue_by_country_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by country\Monthly_sales_by_country.csv',index=False)
monthly_revenue_by_country_df.to_csv(r'C:\Users\SOFYA\OneDrive\Documents\My Tableau Repository\Datasources\cafe project\Revenue by country\Monthly_sales_by_country.csv',index=False)
#aggregating annual revenues by country
revenue_by_country_annually_df=revenue_df.groupby(['country','Year'])['monthly_revenue_total'].sum()
revenue_by_country_annually_df=pd.DataFrame(revenue_by_country_annually_df)
revenue_by_country_annually_df=revenue_by_country_annually_df.reset_index()
revenue_by_country_annually_df=revenue_by_country_annually_df.rename(columns={	'monthly_revenue_total':'annual_revenue_by_country'})

annual_total_df=pd.DataFrame(revenue_by_country_annually_df.groupby(['Year'])['annual_revenue_by_country'].sum())
annual_total_df=annual_total_df.reset_index()

annual_total_df=annual_total_df.rename(columns={'annual_revenue_by_country':'total_annual_revenue'})
revenue_by_country_annually_df['region']=revenue_by_country_annually_df['country'].map(region_dictionary)
revenue_by_country_annually_df=revenue_by_country_annually_df.loc[:,['region','country','Year','annual_revenue_by_country']]
revenue_by_country_annually_df=pd.merge(revenue_by_country_annually_df,annual_total_df,on='Year',how='left')
revenue_by_country_annually_df['percentage_from_total_annual_revenue']=revenue_by_country_annually_df['annual_revenue_by_country']/revenue_by_country_annually_df['total_annual_revenue']*100
revenue_by_country_annually_df['percentage_from_total_annual_revenue']=round(revenue_by_country_annually_df['percentage_from_total_annual_revenue'],2)
revenue_by_country_annually_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by country\Annual_sales_by_country.csv',index=False)

annual_revenue_Argentina_df=revenue_by_country_annually_df[revenue_by_country_annually_df['country']=='Argentina']
annual_revenue_Argentina_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by country\Annual sales Argentina.csv',index=False)
annual_revenue_Peru_df=revenue_by_country_annually_df[revenue_by_country_annually_df['country']=='Peru']
annual_revenue_Peru_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by country\Annual sales Peru.csv',index=False)
annual_revenue_USA_df=revenue_by_country_annually_df[revenue_by_country_annually_df['country']=='USA']
annual_revenue_USA_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by country\Annual sales USA.csv',index=False)



